<a href="https://colab.research.google.com/github/Aunabil4602/Kaggle-Competitions/blob/master/melanoma_pytorch_with_metaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-106ec296-521c-239f-ebfa-6d62dc8c55af)
Mon Aug 17 14:54:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
           

In [ ]:
!pip install -U git+https://github.com/albu/albumentations

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-2guuiiwn
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-2guuiiwn
     |████████████████████████████████| 952kB 8.8MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=66166 sha256=e9b685658d0bc153f4aa5063fcac3cdb3e0dd65fcb9513cfbdaa5a6daad4da61
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2563i2f/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
#imports 

import os 
import shutil 
import numpy as np 
from PIL import Image 
import PIL 
import matplotlib.pyplot as plt 
import pandas as pd 
import math
import time
import sys
import cv2
import random
from sklearn.metrics import roc_auc_score

from __future__ import print_function, division
import torch
import torchvision
import albumentations

from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
shutil.copy('/content/drive/My Drive/data_sets/Melanoma Data/ZipData/train_540.zip','training.zip')
shutil.copy('/content/drive/My Drive/data_sets/Melanoma Data/ZipData/hairs.zip','hairs.zip')
shutil.copy('/content/drive/My Drive/data_sets/Melanoma Data/ZipData/extra_melanoma_540.zip','extra.zip')
shutil.copy('/content/drive/My Drive/data_sets/Melanoma Data/ZipData/train_meta_v1.csv','./meta_data.csv')

os.mkdir('train')
os.mkdir('hairs')

#importing training set
shutil.unpack_archive('training.zip','train','zip')
shutil.unpack_archive('hairs.zip','hairs','zip')
shutil.unpack_archive('extra.zip','./','zip')

csv_v1=pd.read_csv('meta_data.csv')
print(len(csv_v1))


temp_list=os.listdir('resized_540')
for name in temp_list:
  shutil.move('resized_540/'+name,'train/'+name)

all_images=os.listdir('train')
for name in all_images:
    if name[:5]=='M_IMG':
        os.remove('train/'+name)


all_normal=[]
all_melanoma=[]
cnt=0
for it,val in enumerate(csv_v1['target']):

    if val==0:
        all_normal.append(it)
        cnt+=1
    else :
        all_melanoma.append(it)
        
print(len(os.listdir('train')))
print('Total Malignant: ',len(all_normal))
print('Total Melanoma: ',len(all_melanoma))

37648
37648
Total Malignant:  32542
Total Melanoma:  5106


In [ ]:
csv_v1=pd.read_csv('meta_data.csv')
print(len(csv_v1))

all_normal=[]
all_melanoma=[]
cnt=0
for it,val in enumerate(csv_v1['target']):

    if val==0:
        all_normal.append(it)
        cnt+=1
    else :
        all_melanoma.append(it)
        
print(len(os.listdir('train')))
print('Total Malignant: ',len(all_normal))
print('Total Melanoma: ',len(all_melanoma))

37648
37648
Total Malignant:  32542
Total Melanoma:  5106


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
cpu_device = torch.device("cpu") 
print(device) 

cuda:0


In [ ]:
img_size=300

class MelanomaDataset(torch.utils.data.Dataset):
    
    def __init__(self, data_,index_,is_train=True):
        self.dataframe = data_
        self.main_list=index_
        if is_train:
            self.transform = albumentations.core.composition.Compose([
                albumentations.augmentations.transforms.RandomResizedCrop(height=img_size, width=img_size, scale=(0.4, 1.0),p=1.0),
                albumentations.augmentations.transforms.ShiftScaleRotate(rotate_limit=90.0,shift_limit=0.0,scale_limit=0.0,p=1.0),
                albumentations.augmentations.transforms.HorizontalFlip(p=0.5),
                albumentations.augmentations.transforms.VerticalFlip(p=0.5),
                albumentations.augmentations.transforms.HueSaturationValue(hue_shift_limit=20,sat_shift_limit=20,val_shift_limit=20,p=1.0),
                albumentations.augmentations.transforms.RandomBrightnessContrast(brightness_limit=0.3,contrast_limit= 0.3,p=1.0),
                albumentations.augmentations.transforms.FancyPCA(alpha=0.1,p=1.0),
                AdvancedHairAugmentation(p=0.5),
                albumentations.augmentations.transforms.Cutout(num_holes=10, max_h_size=10, max_w_size=10, p=0.5),
                albumentations.augmentations.transforms.Normalize(),
                albumentations.pytorch.transforms.ToTensorV2(),
            ])
        else:
            self.transform = albumentations.core.composition.Compose([ 
                albumentations.augmentations.transforms.Resize(height=344, width=344), 
                albumentations.augmentations.transforms.CenterCrop(height=img_size, width=img_size),                                                     
                albumentations.augmentations.transforms.Normalize(), 
                albumentations.pytorch.transforms.ToTensorV2(),
            ])
            
    def __len__(self):
        return len(self.main_list)
    
    def __getitem__(self, index):
        index=self.main_list[index]
        #print('lol')
        label=self.dataframe['target'][index]
        #print('lol')
        name = self.dataframe['image_name'][index]

        if name[0]=='M' or name[0]=='N':
            if label==0:
                name='N'+name[1:]
            else :
                name='M'+name[1:]
        #print('lol')
        image = cv2.imread('train/'+name,cv2.IMREAD_UNCHANGED)   
        #print(name,label,type(image))
        image = self.transform(image=image)
        image = image['image']

        csv_data = np.array(self.dataframe.iloc[index][['torso','upper extremity','lower extremity',
                                                       'head/neck','palms/soles','oral/genital','sex', 'age']].values, 
                            dtype=np.float32)
        csv_data[-1]=(csv_data[-1]-50.2762)/15.0
        #print('lol')
        return image,csv_data,label



class AdvancedHairAugmentation(ImageOnlyTransform):
    def __init__(self, hairs: int = 10, hairs_folder: str = "hairs" , always_apply=False, p=0.5):
      self.hairs = hairs
      self.hairs_folder = hairs_folder
      super().__init__(always_apply, p)

    def apply(self, img, **params):
        n_hairs = random.randint(0, self.hairs)+5

        if not n_hairs:
            return img

        height, width, _ = img.shape  # target image width and height
        hair_images = [im for im in os.listdir(self.hairs_folder) if 'png' in im]

        for _ in range(n_hairs):
            hair = cv2.imread(os.path.join(self.hairs_folder, random.choice(hair_images)))
            hair = cv2.flip(hair, random.choice([-1, 0, 1]))
            hair = cv2.rotate(hair, random.choice([0, 1, 2]))

            h_height, h_width, _ = hair.shape  # hair image width and height
            roi_ho = random.randint(0, img.shape[0] - hair.shape[0])
            roi_wo = random.randint(0, img.shape[1] - hair.shape[1])
            roi = img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
            hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

            dst = cv2.add(img_bg, hair_fg, dtype=cv2.CV_64F)
            img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

        return img

img_wh=img_size
centre=(img_wh-1)/2.0
rad=(img_wh-2)/2.0
out_x=[]
out_y=[]

for x in range(img_wh):
  for y in range(img_wh):
    dis=math.sqrt((x-centre)*(x-centre)+(y-centre)*(y-centre))
    if dis>rad:
      out_x.append(x)
      out_y.append(y)
out_x=tuple(np.asarray(out_x))
out_y=tuple(np.asarray(out_y))



In [ ]:
#model

!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

class CNN_Meta_NET(torch.nn.Module):
    def __init__(self, model_name, input_size,output_size):
        super().__init__()
        self.input_size, self.output_size = input_size, output_size
        
        self.model_name=model_name
        self.cnn = EfficientNet.from_pretrained(self.model_name,num_classes=1)
        self.cnn._fc=Identity()
        for param in self.cnn.parameters():
            param.requires_grad=False

        self.meta = torch.nn.Sequential(torch.nn.Linear(self.input_size, 200),
                                 torch.nn.BatchNorm1d(200),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(p=0.2))

        self.classification = torch.nn.Linear(1536 + 200, self.output_size) 
        
    def forward(self, image, meta_data, prints=False):    
        
        cnn_out = self.cnn(image)
        meta_out = self.meta(meta_data)
        
        cnn_meta_out = torch.cat((cnn_out, meta_out), dim=1)
        final_out = self.classification(cnn_meta_out)
        
        return final_out

In [ ]:
# setting up model and scheduler

model_name='efficientnet-b3'
model = CNN_Meta_NET(model_name,8,1)
model=model.to(device)

lr_patience=1
lr_factor=0.1
b_size=16
starting_lr=0.1*b_size/256

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=starting_lr, momentum=0.9,weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',patience=lr_patience,min_lr=1e-9, verbose=True, factor=lr_factor)

print(model_name)
print('batch size: ',b_size,' , lr: ',starting_lr)

Loaded pretrained weights for efficientnet-b3
efficientnet-b3
batch size:  16  , lr:  0.00625


In [ ]:

number_of_folds = 4
num_move_normal=int(len(all_normal)/number_of_folds)
num_move_melanoma=int(len(all_melanoma)/number_of_folds)
epochs=15
print(num_move_melanoma,num_move_normal)


for itr_folds in range(4):
    print('------------Running Fold no: ',itr_folds+1,'------------')
 

    ## setting test images
    cnt_mel,cnt_nor= 0,0
    test_data_idx,train_data_idx=[],[]
    for it in range(itr_folds*num_move_normal,(itr_folds==(number_of_folds-1))*len(all_normal)+(itr_folds!=(number_of_folds-1))*(itr_folds+1)*num_move_normal):
        test_data_idx.append(all_normal[it])

    for it in range(itr_folds*num_move_melanoma,(itr_folds==(number_of_folds-1))*len(all_melanoma)+(itr_folds!=(number_of_folds-1))*(itr_folds+1)*num_move_melanoma):
        test_data_idx.append(all_melanoma[it])

    ## setting training images
    for it in range(0,itr_folds*num_move_normal):
        cnt_nor+=1
        train_data_idx.append(all_normal[it])

    for times in range(6):
        for it in range(0,itr_folds*num_move_melanoma):
            cnt_mel+=1
            train_data_idx.append(all_melanoma[it])

    for it in range((itr_folds==(number_of_folds-1))*len(all_normal)+(itr_folds!=(number_of_folds-1))*(itr_folds+1)*num_move_normal,len(all_normal)):
        cnt_nor+=1
        train_data_idx.append(all_normal[it])

    for times in range(6):
        for it in range((itr_folds==(number_of_folds-1))*len(all_melanoma)+(itr_folds!=(number_of_folds-1))*(itr_folds+1)*num_move_melanoma,len(all_melanoma)):
            cnt_mel+=1
            train_data_idx.append(all_melanoma[it])

    print('Train Size :',len(train_data_idx),' (nor: {:d} ,mel: {:d})'.format(cnt_nor,cnt_mel),' Test Size : ',len(test_data_idx))

    # setting up train and test dataloader
    train_data = MelanomaDataset(csv_v1,train_data_idx,is_train=True)                                                                                   
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=b_size, shuffle=True,  num_workers=4)

    test_data = MelanomaDataset(csv_v1,test_data_idx,is_train=False)
    test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=b_size, shuffle=True, num_workers=4)
    
    ##train and validation
    stop_patience=3
    best_val_spec=0.0
    
    
    for ep in range(epochs):
    
        ##train single epoch
        state='train'
        acc=0.0
        running_loss = 0.0

        val_spec=0.0
        avg_time=0.0
        st_time=time.time()

        num_correct=0.0
        total_guesses=0.0
        model.train()
        for i, batch_data in enumerate(train_data_loader, 0):   

            inputs,meta_, labels = batch_data
            if inputs.size()[0]<=1:
                break

            #break
            labels = labels.type(torch.FloatTensor)

            inputs = inputs.to(device)
            meta_ = meta_.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs,meta_)
            outputs=outputs.squeeze(1)


            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 

            outputs= torch.sigmoid(outputs)
            num_correct += torch.eq(torch.round(outputs), labels).sum().item()
            total_guesses += inputs.size()[0] 
            running_loss += loss.item()

            print('\rTime: [{:.3f}]s, Epoch: [{:d}/{:d}], Batch: [{:d}/{:d}], Loss: [{:.7f}], Acc: [{:.7f}]'.format(time.time()-st_time,(ep+1),epochs,(i+1),len(train_data_loader),running_loss/(i+1),1.0*num_correct/np.maximum(total_guesses,1.0)),end='',flush=True)
        
        #break
        print('')
        acc=1.0*num_correct/np.maximum(total_guesses,1.0)
        running_loss/=(i+1)

        #validation 
        state='test'
        v_acc=0.0
        v_running_loss = 0.0
        TP_FN=0.0
        TP=0.0
        
        num_correct=0.0
        total_guesses=0.0
        model.eval()
        outs,tars=[],[]
        for i, batch_data in enumerate(test_data_loader, 0):

            inputs,meta_, labels = batch_data
            #inputs[:,:,out_x,out_y]=0.0
            labels=labels.type(torch.FloatTensor)

            inputs = inputs.to(device)
            meta_ = meta_.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs,meta_)
            outputs=outputs.squeeze(1)

            loss = criterion(outputs, labels)

            outputs= torch.sigmoid(outputs)
            num_correct += torch.eq(torch.round(outputs), labels).sum().item()
            total_guesses += inputs.size()[0] 
            v_running_loss += loss.item()

            print('\rValidating => Batch: {:d}/{:d}'.format((i+1),len(test_data_loader)),end='',flush=True)

            for out_val,tar_val in zip(outputs,labels):
                outs.append(out_val.detach().cpu())
                tars.append(tar_val.detach().cpu())
                if torch.round(out_val) == 1:
                    TP_FN +=1
                    if torch.round(tar_val) ==1:
                        TP +=1

        print('\r',end='',flush=True)

        v_acc=1.0*num_correct/total_guesses
        v_running_loss/=(i+1)
        
        outs=np.asarray(outs)
        tars=np.asarray(tars)
        val_spec=roc_auc_score(tars,outs)
        sensitivity=1.0*TP/TP_FN
        
        ## scheduler calling
        scheduler.step(val_spec)
        
        if best_val_spec < val_spec:
            best_val_spec=val_spec
            best_model_dict=model.state_dict()
            stop_patience=np.maximum(stop_patience,2)
        else :
            stop_patience -=1
            if stop_patience ==0:
                print('Val_loss: [{:.7f}], val_acc: [{:.7f}], sensitivity: [{:.7f}], roc: [{:.7f}], best roc: [{:.7f}]'.format(v_running_loss,v_acc,sensitivity,val_spec,best_val_spec))
                print('early stopping !! as no increase in val_spec')
                break
        print('Val_loss: [{:.7f}], val_acc: [{:.7f}], sensitivity: [{:.7f}], roc: [{:.7f}], best roc: [{:.7f}]'.format(v_running_loss,v_acc,sensitivity,val_spec,best_val_spec))
    #break  
    ## saving best model after each fold  
    model.load_state_dict(best_model_dict)  
    torch.save(best_model_dict, model_name+'_'+str(img_size)+'x'+str(img_size)+'_Fold_'+str(itr_folds+1)+' _roc_%.7f'%(best_val_spec)+'_.pth')
    print('****** Best ROC at fold: [{:d}] is : [{:.7f}] ******'.format(itr_folds+1,best_val_spec))
    print('model saved!! for fold:{:d}'.format(int(itr_folds+1)))

1276 8135
------------Running Fold no:  1 ------------
Train Size : 47387  (nor: 24407 ,mel: 22980)  Test Size :  9411
Time: [901.562]s, Epoch: [1/15], Batch: [2962/2962], Loss: [0.3666562], Acc: [0.8402093]
Val_loss: [0.3460749], val_acc: [0.8602699], sensitivity: [0.4862579], roc: [0.8307207], best roc: [0.8307207]
Time: [905.947]s, Epoch: [2/15], Batch: [2962/2962], Loss: [0.3571638], Acc: [0.8466457]
Val_loss: [0.3577486], val_acc: [0.8493253], sensitivity: [0.4520270], roc: [0.8186834], best roc: [0.8307207]
Time: [908.326]s, Epoch: [3/15], Batch: [2962/2962], Loss: [0.3500195], Acc: [0.8495790]
Epoch     3: reducing learning rate of group 0 to 6.2500e-04.
Val_loss: [0.3174783], val_acc: [0.8842843], sensitivity: [0.6130593], roc: [0.8239053], best roc: [0.8307207]
Time: [905.588]s, Epoch: [4/15], Batch: [2962/2962], Loss: [0.3377966], Acc: [0.8553823]
Val_loss: [0.3301175], val_acc: [0.8690894], sensitivity: [0.5176565], roc: [0.8306049], best roc: [0.8307207]
early stopping !! a

KeyboardInterrupt: ignored

In [ ]:
check=os.listdir('train')

In [ ]:
for it in check:
    if it == 'M_ISIC_1616031.jpg':
        print('lol')

lol


In [ ]:
shutil.copy('/content/efficientnet-b3_300x300_Fold_5 _roc_0.9992506_.pth','/content/drive/My Drive/data_sets/Melanoma Data/ZipData/efficientnet-b3_300x300_Fold_5 _roc_0.9992506_.pth')

'/content/drive/My Drive/data_sets/Melanoma Data/ZipData/efficientnet-b3_300x300_Fold_5 _roc_0.9992506_.pth'

In [ ]:
shutil.copy('/content/drive/My Drive/data_sets/Melanoma Data/ZipData/efficientnet-b3_300x300_Fold_4 _roc_0.9996615_.pth','/content/efficientnet-b3_300x300_Fold_4 _roc_0.9996615_.pth')

'/content/efficientnet-b3_300x300_Fold_4 _roc_0.9996615_.pth'

In [ ]:
model.load_state_dict(torch.load('/content/efficientnet-b3_300x300_Fold_4 _roc_0.9996615_.pth'))

<All keys matched successfully>

In [ ]:
temp1=os.listdir('./')
cnt=0
for name in temp1:
  if name[:3]=='eff':
    cnt+=1
    if cnt==3:
      ##shutil.copy('./'+name,'/content/drive/My Drive/data_sets/Melanoma Data/ZipData/'+name)
      print(name)
      break

efficientnet-b3_300x300_Fold_1 _roc_0.9873713_.pth
